In [1]:
#######################################################
# SYSTEM
import os
import sys
#######################################################
# MATH
import numpy as np
import healpy as hp
import pandas as pd
import astropy.io.fits as fits
#######################################################
# TIMERS
import progressbar
import time
#######################################################
# ALESSANDRO/ISABELLA
sys.path.insert(1, '/home/luiz/IC/Codes/Noise_Debias/scripts')
import gmca4im_lib2 as g4i
import Extension4BINGO as cs
#######################################################
from IPython.display import clear_output
import warnings
warnings.filterwarnings("ignore")
#######################################################

In [2]:
#######################################################
# GENERAL
n_realization    = 8          # Number of maps to be produced on the analysis, from a total known number of maps
method           = "ICA"      # Component Separation Method to be used
wtransform       = "identity" # Wavelet transform to be used
maps_wout_mean   = True       # A parameter for centering data with mean = 0

apply_mask       = False      # A parameter to ica sklearn function
add_noise        = False      # A parameter to ica sklearn function
#######################################################


#######################################################
# WAVELET PARAMETERS

# The main aim is to have:

# STARLETS
J     = 1       # Number of scales
use_c = True    # If you will use wavelet scale in the analysis

# S2LET
# If you to use wtransforms by S2Let code, please, fill in the variables below:
L        = None # If you write "None", it will use L=3*nside
J_min    = 1    #
B        = 5    #   
N        = 3    # Number of directions (This is for Directional only)
spin     = 0    # Set to 0 for temperature. if non-zero, plotting routines must be changed! (This is for Directional only)
upsample = 0    # 1 means all scales at full resolution L # 0 means multiresolution wavelet transform (This is for Directional only)
# In the S2LET code, J scales is defined by code and not by J above.

# PYWAVELETS
Jpwt = 2 # Number of scales
pywttype = "db1" 

# CURVELETS

# COUNTURLETS

# SHEARLETS

# RIDGELETS
#######################################################


#######################################################
# COMPONENT SEPARATION PARAMETERS

# ICA
n_s              = 2          #Number of sources to be estimated
whiten           = True  ######## Maintain True
fun              = 'logcosh' # exp,logcosh or tanh
max_iter         = 100 
tol              = 0.0001

# GMCA
mints = 0.05 # min threshold (what is sparse compared to noise?)
nmax  = 100 # number of iterations (usually 100 is safe)
L0    = 0   # switch between L0 norm (1) or L1 norm (0)

AInit     = None
ColFixed  = None
whitening = False
epsi      = 1e-3
verbose   = False
# GMCAExtension
div          = 1 #  J+1  #J/div will should be even number
without_covx = True # if your mixmatrix estimated will use covariance matrix of the observer data with ponderation
#######################################################


#######################################################
# DEBIAS PARAMETERS
seed_used = 10

#######################################################
# PATH PARAMETERS
pathout       = "/home/luiz/IC/Datas_Maps/Cls/NOVOCODIGO_TESTE" #Put here your path to the output cls
cl_type_save  = "reconstruction" #You should choice between reconstruction or residuals cls values

#######################################################
# NAME FILES PARAMETERS
# Name of FITS files inside of the pathmaps
name_mask = "Mask_tot256.fits" #put this file in the same directory of the other maps
#Directory names
dir_observed  = "/home/luiz/IC/Datas_Maps/Cubos_Input_L10_L25_White_Noise"
dir_mask      = "/home/luiz/IC/Datas_Maps/Mask"
dir_prior     = "/home/luiz/IC/Datas_Maps/Cubos_Prior_WN" 
dir_noise     = "/home/luiz/IC/Datas_Maps/wn_masked"         #Put here directory name of the noise maps 
dir_pure      = "/home/luiz/IC/Datas_Maps/Cubos_21cm_Masked" #Put here directory name of the pure maps 
dir_projprior = "/home/luiz/IC/Datas_Maps/Cubos_Prior_WN"    #Put here directory name of the prior maps
dir_projnoise = "/home/luiz/IC/Datas_Maps/wn_masked"         #Put here directory name of the noise maps
dir_projpure  = "/home/luiz/IC/Datas_Maps/Cubos_21cm_Masked" #Put here directory name of the pure maps
#######################################################

In [3]:
#######################################################
# ?
params_maps = pd.Series({"without_mean":maps_wout_mean, "apply_mask":apply_mask, "add_noise":add_noise, "cl_type_save":cl_type_save})
params_CS   = pd.Series({"method":method,
                         "A_ini":AInit, "ColFixed":ColFixed, "whitening":whitening, "epsi":epsi, "verbose":verbose, "ns":n_s, "mints":mints,"nmax":nmax, "L0":L0, "division":div, "without_covx":without_covx, "seed_used":seed_used,
                         "whiten":whiten, "fun":fun, "max_iter":max_iter, "tol":tol})
params_WT   = pd.Series({"wtransform":np.asarray(wtransform), "use_c":use_c, "J":J, 
                         "L":L, "Jmin":J_min, "B": B, "N":N, "spin":spin, "upsample":upsample,
                         "Jpwt":Jpwt, "pywttype":pywttype.lower()})
params_path = pd.Series({"pathout":pathout, "dir_observed":dir_observed, "dir_mask":dir_mask, "dir_noise":dir_noise, "dir_prior":dir_prior,"dir_pure":dir_pure, "name_mask":name_mask})
#######################################################

In [4]:
#  FOREGROUNDS + HI + MIXMATRIX

In [5]:
map_names = np.asarray(os.listdir(params_path.dir_observed))                  # Goes to directory with observed maps and puts their names on an array of strings
map_names = np.random.choice(map_names, size = n_realization, replace=False)  # Randomically choose "n" maps from a total "m", n<m
nseed_0 = cs.extracting_seed_from_filenames(vectornames=map_names)            # Create vector with the number labels of maps. Map1 = seed 1

params_maps["getdata"] = "observed"  #?
subdirs = cs.checkdir(params_path.pathout, subdirs=["21cm","foregrounds","mixmatrix"]) # Create the directories for outputs

timei = time.time()
bar = progressbar.ProgressBar(maxval=map_names.size)
for i,iname in enumerate(map_names):  # Loop that does Component Separation Analysis. number of loops = length of map_names
    clear_output(wait=True) #??
    bar.update(i)
    time0 = time.time() #?
    params_path["name_observed"] = iname # aqui começa a análise de component separation e wavelets, 
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    X = cs.maps2CSmaps(X, params_WT, params_CS)         # chegando nesse ponto, temos as matrizes já com os Cls, portanto já temos o espectro de potencia após passar pelo ICA
    params_maps["iseed"]="L"+str(nseed_0[i])
    cs.saveouts(mrec=X, params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)
    del X
    time0   = time.time()-time0 #?
clear_output(wait=True)
bar.finish()

100% (8 of 8) |##########################| Elapsed Time: 0:01:26 Time:  0:01:26


In [6]:
#  NOISE

In [7]:
map_names = np.asarray(os.listdir(params_path.dir_noise))                  # Same for different directory
nseed_1 = cs.extracting_seed_from_filenames(vectornames=map_names)         # Extract new seeds
index=[] 

for i in range(len(nseed_0)):                        # This loop will make sure that we get only the seeds from the first box, nseed_0. (we avoid the situation where we have the map with seed 15 in one directory (observed) but not in another (proj pure))
    n0 = nseed_0[i]
    index.append(np.where(nseed_1==n0)[0][0])
index = np.asarray(index)
map_names = map_names[index]
nseed_1 = nseed_1[index]

params_maps["getdata"] = "noise" # Same for different directory
subdirs = ["noise"]              # Same for different directory
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):     #This will check if there is a directory, delete and replace if it already exists
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))
    
timei   = time.time()
bar = progressbar.ProgressBar(maxval=map_names.size)
for i,iname in enumerate(map_names):
    clear_output(wait=True)
    bar.update(i)
    time0 = time.time()
    params_path["name_noise"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed_1[i])
    cs.saveouts(mrec=X, params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    

bar.finish()

100% (8 of 8) |##########################| Elapsed Time: 0:00:23 Time:  0:00:23


In [8]:
# PRIOR

In [9]:
map_names = np.asarray(os.listdir(params_path.dir_prior))                  # Same for different directory
nseed_1 = cs.extracting_seed_from_filenames(vectornames=map_names)         # Extract new seeds
index=[] 

for i in range(len(nseed_0)):                        # This loop will make sure that we get only the seeds from the first box, nseed_0. (we avoid the situation where we have the map with seed 15 in one directory (observed) but not in another (proj pure))
    n0 = nseed_0[i]
    index.append(np.where(nseed_1==n0)[0][0])
index = np.asarray(index)
map_names = map_names[index]
nseed_1 = nseed_1[index]

params_maps["getdata"] = "prior" # Same for different directory
subdirs = ["prior"]              # Same for different directory
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):     #This will check if there is a directory, delete and replace if it already exists
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))
    
timei   = time.time()
bar = progressbar.ProgressBar(maxval=map_names.size)
for i,iname in enumerate(map_names):
    clear_output(wait=True)
    bar.update(i)
    time0 = time.time()
    params_path["name_prior"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed_1[i])
    cs.saveouts(mrec=X, params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    

bar.finish()

100% (8 of 8) |##########################| Elapsed Time: 0:00:22 Time:  0:00:22


In [10]:
# 21CM PURE

In [11]:
map_names = np.asarray(os.listdir(params_path.dir_pure))                  # Same for different directory
nseed_1 = cs.extracting_seed_from_filenames(vectornames=map_names)         # Extract new seeds
index=[] 

for i in range(len(nseed_0)):                        # This loop will make sure that we get only the seeds from the first box, nseed_0. (we avoid the situation where we have the map with seed 15 in one directory (observed) but not in another (proj pure))
    n0 = nseed_0[i]
    index.append(np.where(nseed_1==n0)[0][0])
index = np.asarray(index)
map_names = map_names[index]
nseed_1 = nseed_1[index]

params_maps["getdata"] = "pure" # Same for different directory
subdirs = ["pure"]              # Same for different directory
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):     #This will check if there is a directory, delete and replace if it already exists
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))
    
timei   = time.time()
bar = progressbar.ProgressBar(maxval=map_names.size)
for i,iname in enumerate(map_names):
    clear_output(wait=True)
    bar.update(i)
    time0 = time.time()
    params_path["name_pure"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed_1[i])
    cs.saveouts(mrec=X, params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    

bar.finish()

100% (8 of 8) |##########################| Elapsed Time: 0:00:23 Time:  0:00:23


In [12]:
#  21CM PROJ PURE

In [13]:
map_names = np.asarray(os.listdir(params_path.dir_pure))                  # Same for different directory
nseed_1 = cs.extracting_seed_from_filenames(vectornames=map_names)         # Extract new seeds
index=[] 

for i in range(len(nseed_0)):                        # This loop will make sure that we get only the seeds from the first box, nseed_0. (we avoid the situation where we have the map with seed 15 in one directory (observed) but not in another (proj pure))
    n0 = nseed_0[i]
    index.append(np.where(nseed_1==n0)[0][0])
index = np.asarray(index)
map_names = map_names[index]
nseed_1 = nseed_1[index]

params_maps["getdata"] = "pure" # Same for different directory
subdirs = ["projpure"]              # Same for different directory

L0      = "L{}".format(params_CS.seed_used)
A       = cs.loadmixmatrix(params_path.pathout,"mixmatrix")
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):     #This will check if there is a directory, delete and replace if it already exists
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))
    
timei   = time.time()
bar = progressbar.ProgressBar(maxval=map_names.size)
for i,iname in enumerate(map_names):
    clear_output(wait=True)
    bar.update(i)
    time0 = time.time()
    params_path["name_pure"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed_1[i])
    cs.saveouts(mrec=X, A=A[L0], params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    

bar.finish()

100% (8 of 8) |##########################| Elapsed Time: 0:00:25 Time:  0:00:25


In [14]:
# 21CM PROJ NOISE

In [15]:
map_names = np.asarray(os.listdir(params_path.dir_noise))                  # Same for different directory
nseed_1 = cs.extracting_seed_from_filenames(vectornames=map_names)         # Extract new seeds
index=[] 

for i in range(len(nseed_0)):                        # This loop will make sure that we get only the seeds from the first box, nseed_0. (we avoid the situation where we have the map with seed 15 in one directory (observed) but not in another (proj pure))
    n0 = nseed_0[i]
    index.append(np.where(nseed_1==n0)[0][0])
index = np.asarray(index)
map_names = map_names[index]
nseed_1 = nseed_1[index]

params_maps["getdata"] = "noise" # Same for different directory
subdirs = ["projnoise"]              # Same for different directory

L0      = "L{}".format(params_CS.seed_used)
A       = cs.loadmixmatrix(params_path.pathout,"mixmatrix")
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):     #This will check if there is a directory, delete and replace if it already exists
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))
    
timei   = time.time()
bar = progressbar.ProgressBar(maxval=map_names.size)
for i,iname in enumerate(map_names):
    clear_output(wait=True)
    bar.update(i)
    time0 = time.time()
    params_path["name_noise"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed_1[i])
    cs.saveouts(mrec=X, A=A[L0], params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    

bar.finish()

100% (8 of 8) |##########################| Elapsed Time: 0:00:25 Time:  0:00:25


In [16]:
# 21CM PROJ PRIOR

In [17]:
map_names = np.asarray(os.listdir(params_path.dir_prior))                  # Same for different directory
nseed_1 = cs.extracting_seed_from_filenames(vectornames=map_names)         # Extract new seeds
index=[] 

for i in range(len(nseed_0)):                        # This loop will make sure that we get only the seeds from the first box, nseed_0. (we avoid the situation where we have the map with seed 15 in one directory (observed) but not in another (proj pure))
    n0 = nseed_0[i]
    index.append(np.where(nseed_1==n0)[0][0])
index = np.asarray(index)
map_names = map_names[index]
nseed_1 = nseed_1[index]

params_maps["getdata"] = "prior" # Same for different directory
subdirs = ["projprior"]              # Same for different directory

L0      = "L{}".format(params_CS.seed_used)
A       = cs.loadmixmatrix(params_path.pathout,"mixmatrix")
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):     #This will check if there is a directory, delete and replace if it already exists
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))
    
timei   = time.time()
bar = progressbar.ProgressBar(maxval=map_names.size)
for i,iname in enumerate(map_names):
    clear_output(wait=True)
    bar.update(i)
    time0 = time.time()
    params_path["name_prior"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed_1[i])
    cs.saveouts(mrec=X, A=A[L0], params_path=params_path, params_maps=params_maps, params_WT=params_WT, params_CS=params_CS, subdirs=subdirs)    

bar.finish()

100% (8 of 8) |##########################| Elapsed Time: 0:00:29 Time:  0:00:29
